In [4]:
# 1. Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start Ollama server
import subprocess
import time
import os
os.environ['PATH'] += ':/usr/local/bin'

with open("ollama_server.log", "w") as log_file:
    process = subprocess.Popen(["ollama", "serve"], stdout=log_file, stderr=log_file)

time.sleep(15) # Giving the L4 server time to warm up

# 3. Pull Llama 3 (8B) - The best model for this task
!ollama pull llama3
!pip install ollama

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.



In [5]:
from google.colab import drive
import os
import ollama
import re
import json

drive.mount('/content/drive')

# UPDATE THESE PATHS to your specific Dissertation folders
BASE_DIR = "/content/drive/MyDrive/Dissertation"
TRANSCRIPT_DIR = os.path.join(BASE_DIR, "transcripts_txt1")
SUMMARY_DIR = os.path.join(BASE_DIR, "summaries_json1")
PROMPT_FILE = os.path.join(BASE_DIR, "updatepromotMeeting.txt")

os.makedirs(TRANSCRIPT_DIR, exist_ok=True)
os.makedirs(SUMMARY_DIR, exist_ok=True)

# Load your improved prompt file
with open(PROMPT_FILE, 'r') as f:
    base_context = f.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:

# Define your 8 topics for Story 1
TOPICS = [
    "Story 1 Kick-off: Route Creation UI & Schema Planning.",
    "PES Review: Technical Specification alignment & Code-Drop sync.",
    "Bug Triage I: Validation issues & MQL Schema limitations.",
    "Bug Triage II: Regression testing & Use-case disputes.",
    "Build Sync: Deploying Fix Build v1.1 to Validation.",
    "Final Validation: Closing UTP/HLT and UI Verification.",
    "Sprint Governance: Story Closure & Risk Assessment.",
    "Retrospective: Lessons learned & Technical conflicts."
]

In [7]:
import ollama, re, json


def save_files(m_id, transcript, json_raw):
    """Cleans and saves the transcript and JSON files."""
    # 1. Save Transcript
    with open(f"{TRANSCRIPT_DIR}/{m_id}_transcript.txt", "w", encoding="utf-8") as f:
        f.write(transcript)

    # 2. Clean and Save JSON
    # Use Regex to find the JSON block in case the AI added extra text
    json_match = re.search(r'\{.*\}', json_raw, re.DOTALL)
    if json_match:
        try:
            json_data = json.loads(json_match.group())
            with open(f"{SUMMARY_DIR}/{m_id}_summary.json", "w", encoding="utf-8") as f:
                json.dump(json_data, f, indent=4)
            print(f"Successfully saved {m_id}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON for {m_id}. Saving raw output as backup.")
            with open(f"{SUMMARY_DIR}/{m_id}_ERROR.txt", "w") as f:
                f.write(json_raw)
    else:
        print(f"Error: No JSON found in response for {m_id}")

def generate_phase1_meeting(i, topic):
    m_num = i + 1
    m_id = f"Meeting_{m_num:02d}"

    # --- PHASE 1 DYNAMIC PROMPTING ---
    if m_num == 1:
        instr = "STORY KICKOFF: Story Points (Planning Poker). Tasks: PES, Impl, UTP, HLT, Validation. No Bugs."
    elif m_num == 2:
        instr = "PES REVIEW: Technical alignment. Set Code-Drop date. No Bugs."
    elif m_num in [3, 4]:
        instr = "BUG TRIAGE: Discuss Bugs. Debate 'Not a Bug/Invalid Use Case'. Assign to Dev."
    elif m_num in [5, 6]:
        instr = "BUILD SHARING: Handover fix build to QE. Status check on Tasks. No new Stories."
    elif m_num == 7:
        instr = "STORY CLOSURE: Review DoD. Sarah closes Story. Identify Sprint Risks."
    elif m_num == 8:
        instr = "RETROSPECTIVE: Lessons learned. Process conflicts. No Tasks/Bugs. Use keys 'takeaways' and 'conflicts'."

    # STEP 1: TRANSCRIPT
    #t_prompt = f"Using Context from promotMeeting.txt: TOPIC: {topic}. TYPE: {instr}. Generate 2000+ words."
    t_prompt = f"""
    {base_context}
    CURRENT MEETING TOPIC: {topic}
    MEETING TYPE: {instr}

    TASK: Generate a 2000+ word verbatim technical transcript.
    """

    print(f"Starting {m_id}...")

    t_res = ollama.generate(model='llama3', prompt=t_prompt)
    transcript = t_res['response'].strip()

    # STEP 2: JSON
    s_prompt = f"Based on this transcript: {transcript}\n\nExtract JSON. Meeting {m_num} rules apply. Return ONLY JSON."
    s_res = ollama.generate(model='llama3', prompt=s_prompt, options={"temperature": 0.1})

    save_files(m_id, transcript, s_res['response'])

# Run the generation
for i, topic in enumerate(TOPICS):
    generate_phase1_meeting(i, topic)

Starting Meeting_01...
Error: Could not decode JSON for Meeting_01. Saving raw output as backup.
Starting Meeting_02...
Successfully saved Meeting_02
Starting Meeting_03...
Successfully saved Meeting_03
Starting Meeting_04...


KeyboardInterrupt: 

In [3]:
def generate_meeting_files(i, topic):
    meeting_id = f"Meeting_{i+1:02d}"
    print(f"--- Processing {meeting_id} ---")

    # STEP 1: GENERATE TRANSCRIPT (Conversational & SDLC Focused)
    t_prompt = f"""{base_context}
    TASK: Write a 2000-word verbatim technical transcript for {topic}.

    PHASE: Design & Development (No Bugs).

    CRITICAL DIALOGUE INSTRUCTION:
    The meeting must naturally conclude with Sarah (PO) assigning these specific engineering tasks:
    1. Vikram to work on the Performance & Engineering Specification (PES).
    2. Anjali to handle the Development Implementation.
    3. Vikram/Anjali to define the Unit Test Plan (UTP).
    4. Priya/Mark to prepare the High-Level Testing (HLT) scenarios.

    Sarah should speak these in full, professional sentences.
    DO NOT use robotic labels or mention bugs."""

    t_res = ollama.generate(model='llama3', prompt=t_prompt)
    transcript = t_res['response'].strip()

    # Save Transcript
    with open(f"{TRANSCRIPT_DIR}/{meeting_id}_transcript.txt", "w", encoding="utf-8") as f:
        f.write(transcript)

    # STEP 2: JSON EXTRACTION (Updating Summary with PES/UTP/HLT)
    s_prompt = f"""
    Read the transcript and create a detailed JSON.

    STRICT MAPPING RULES:
    - If the task is about specifications, summary must start with "PES: [Detail]"
    - If the task is about unit testing, summary must start with "UTP: [Detail]"
    - If the task is about validation/test cases, summary must start with "HLT: [Detail]"
    - If the task is about coding, summary must start with "Implementation: [Detail]"
    - The 'issue_type' for all these should be 'Task'.
    - DO NOT include 'Bug' tasks.

    JSON SCHEMA:
    {{
      "meeting_id": "{meeting_id}",
      "project": "Enovia Route Automation",
      "summary": ["Detailed sentences on technical decisions"],
      "action_items": [
          {{
            "summary": "PES/UTP/HLT/Implementation: [Task Description]",
            "description": "Full technical details based on Sarah's words",
            "assignee": "Name",
            "issue_type": "Task",
            "story_points": 3,
            "due_date": "2025-01-30"
          }}
      ]
    }}
    RETURN ONLY JSON."""

    s_res = ollama.generate(model='llama3', prompt=s_prompt, options={"temperature": 0.1})

    # Extract and save JSON
    json_match = re.search(r'\{.*\}', s_res['response'], re.DOTALL)
    if json_match:
        try:
            valid_json = json.loads(json_match.group())
            with open(f"{SUMMARY_DIR}/{meeting_id}_summary.json", "w", encoding="utf-8") as f:
                json.dump(valid_json, f, indent=4)
            print(f"Success: {meeting_id} with PES/UTP/HLT labeling.")
        except:
            print(f"Error: JSON parsing failed.")

In [4]:
topics = [
   "Meeting 01: Project Kick-off: Scope definition for Route Creation and UI requirements.",
   "Meeting 02: Business Requirements: Defining 'Route Base Purpose' and mandatory attributes."
  ]

In [5]:
# Run for all topics
for i, topic in enumerate(topics):
    generate_meeting_files(i, topic)

--- Processing Meeting_01: Meeting 01: Project Kick-off: Scope definition for Route Creation and UI requirements. ---
Done: Meeting_01 (Transcript + JSON)
--- Processing Meeting_02: Meeting 02: Business Requirements: Defining 'Route Base Purpose' and mandatory attributes. ---
Done: Meeting_02 (Transcript + JSON)


# Synthetic Meeting Data Example
This notebook demonstrates how to structure both meeting transcripts and summaries for route creation UI project test data.

## Meeting Transcript Example
Below is a sample transcript for a meeting between QE and Dev about route creation.

In [ ]:
# Sample transcript for Meeting 05
meeting_05_transcript = '''
[QE]: We need to add a new route for the delivery module. Can you create the initial API endpoints?
[Dev]: Sure, I’ll start with the POST and GET endpoints for route creation and retrieval.
[QE]: Please ensure the route validation logic is included. Also, add unit tests for edge cases.
[Dev]: Noted. I’ll create the validation and add tests. Should I create a Jira story for this?
[QE]: Yes, create a story and link the tasks. If you find any blockers, log them as bugs.
[Dev]: Understood. I’ll update the Jira board after the initial commit.
'''
print(meeting_05_transcript)

## Meeting Summary Example
Below is a sample summary for Meeting 05, showing how to create, close, or update tasks and bugs.

In [ ]:
# Sample summary for Meeting 05
meeting_05_summary = {
    "meeting_id": "05",
    "project": "RouteCreationUI",
    "action_items": [
        {
            "type": "story",
            "title": "Create delivery route API endpoints",
            "status": "open",
            "tasks": [
                {
                    "title": "Implement POST endpoint for route creation",
                    "status": "open"
                },
                {
                    "title": "Implement GET endpoint for route retrieval",
                    "status": "open"
                },
                {
                    "title": "Add route validation logic",
                    "status": "open"
                },
                {
                    "title": "Add unit tests for edge cases",
                    "status": "open"
                }
            ]
        },
        {
            "type": "bug",
            "title": "Blocker: Route validation fails for certain edge cases",
            "status": "open",
            "details": "Log bug if validation logic does not handle all scenarios."
        }
    ]
}
import json
print(json.dumps(meeting_05_summary, indent=2))